# Imports

In [1]:
import logging
import os

import pandas as pd
from typing import List

import helpers.dbg as hdbg
import helpers.env as henv
import helpers.printing as hprint
import helpers.s3 as hs3
import im.cryptodatadownload.data.load.loader as imcdalolo
import im_v2.ccxt.data.client.clients as imvcdclcl
import im_v2.ccxt.universe.universe as imvccunun
import im_v2.common.data.client.clients as ivcdclcl

In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-efbbb9c0-690b-4048-8d76-aa014e26faf0.json'
>>ENV<<: is_inside_container=True: code_version=cmamp-1.0.3, container_version=cmamp-1.0.3, is_inside_docker=True, is_inside_ci=False, CI_defined=True, CI=''
>>ENV<<: AM_AWS_PROFILE=True AM_ECR_BASE_PATH=True AM_S3_BUCKET=True AM_TELEGRAM_TOKEN=True AWS_ACCESS_KEY_ID=False AWS_DEFAULT_REGION=False AWS_SECRET_ACCESS_KEY=False GH_ACTION_ACCESS_TOKEN=True
# Git
    branch_name='CMTask324_Extended_analysis_of_CDD'
    hash='28608f59c'
    # Last commits:
      * 28608f59c Maksim Sergeychikov CMTask324: Address most comments                                  (  23 hours ago) Sun Dec 19 18:09:55 2021  (HEAD -> CMTask324_Extended_analysis_of_CDD, origin/CMTask324_Extended_analysis_of_CDD)
      * b779cfd0f Maksim Sergeychikov CMTask324: Update CCXT client                                     (    4 days ago) Thu Dec 16 19:36:29 2021        

# Load the data universe

## CCXT

In [3]:
ccxt_universe = imvccunun.get_vendor_universe(version="v03")

## CDD

In [4]:
universe_cdd = imvccunun.get_vendor_universe(version="v01", vendor="CDD")

# Compare universes

In [5]:
_LOG.info("Number of full symbols in CCXT: %s", len(ccxt_universe))
_LOG.info("Number of full symbols in CDD: %s", len(universe_cdd))

Number of full symbols in CCXT: 38
Number of full symbols in CDD: 117


In [6]:
# Intersection of full symbols between two vendors.
currency_pair_intersection = set(ccxt_universe).intersection(universe_cdd)
_LOG.info("Number of similar full symbols: %s", len(currency_pair_intersection))
display(currency_pair_intersection)

Number of similar full symbols: 18


{'binance::ADA_USDT',
 'binance::BNB_USDT',
 'binance::BTC_USDT',
 'binance::EOS_USDT',
 'binance::ETH_USDT',
 'binance::LINK_USDT',
 'binance::SOL_USDT',
 'ftx::BNB_USDT',
 'ftx::BTC_USDT',
 'ftx::ETH_USDT',
 'ftx::LINK_USDT',
 'ftx::XRP_USDT',
 'kucoin::ADA_USDT',
 'kucoin::BNB_USDT',
 'kucoin::BTC_USDT',
 'kucoin::EOS_USDT',
 'kucoin::ETH_USDT',
 'kucoin::XRP_USDT'}

In [7]:
# Full symbols that are included in CCXT but not in CDD.
ccxt_and_not_cdd = set(ccxt_universe).difference(universe_cdd)
_LOG.info(
    "Number of full symbols that are included in CCXT but not in CDD: %s",
    len(ccxt_and_not_cdd),
)
display(ccxt_and_not_cdd)

Number of full symbols that are included in CCXT but not in CDD: 20


{'binance::AVAX_USDT',
 'binance::DOGE_USDT',
 'ftx::DOGE_USDT',
 'ftx::SOL_USDT',
 'gateio::ADA_USDT',
 'gateio::AVAX_USDT',
 'gateio::BNB_USDT',
 'gateio::BTC_USDT',
 'gateio::DOGE_USDT',
 'gateio::EOS_USDT',
 'gateio::ETH_USDT',
 'gateio::FIL_USDT',
 'gateio::LINK_USDT',
 'gateio::SOL_USDT',
 'gateio::XRP_USDT',
 'kucoin::AVAX_USDT',
 'kucoin::DOGE_USDT',
 'kucoin::FIL_USDT',
 'kucoin::LINK_USDT',
 'kucoin::SOL_USDT'}

In [8]:
# Full symbols that are included in CDD but not in CCXT.
cdd_and_not_ccxt = set(universe_cdd).difference(ccxt_universe)
_LOG.info(
    "Number of full symbols that are included in CDD but not in CCXT: %s",
    len(cdd_and_not_ccxt),
)
display(cdd_and_not_ccxt)

Number of full symbols that are included in CDD but not in CCXT: 99


{'binance::AAVE_USDT',
 'binance::BAT_USDT',
 'binance::BTT_USDT',
 'binance::CELR_USDT',
 'binance::CVC_USDT',
 'binance::DAI_USDT',
 'binance::DASH_USDT',
 'binance::DOT_USDT',
 'binance::ETC_USDT',
 'binance::FIL_USDT',
 'binance::ICP_USDT',
 'binance::ICX_USDT',
 'binance::LRC_USDT',
 'binance::LTC_USDT',
 'binance::MATIC_USDT',
 'binance::MKR_USDT',
 'binance::NEO_USDT',
 'binance::ONE_USDT',
 'binance::PAX_USDT',
 'binance::QTUM_USDT',
 'binance::SCU_USDT',
 'binance::TRX_USDT',
 'binance::TUSD_USDT',
 'binance::UNI_USDT',
 'binance::USDC_USDT',
 'binance::VET_USDT',
 'binance::XLM_USDT',
 'binance::XMR_USDT',
 'binance::XRP_USDT',
 'binance::ZEC_USDT',
 'bitfinex::AID_USD',
 'bitfinex::BAT_USD',
 'bitfinex::BTC_EUR',
 'bitfinex::BTC_GBR',
 'bitfinex::BTC_JPY',
 'bitfinex::BTC_USD',
 'bitfinex::DAI_BTC',
 'bitfinex::DAI_USD',
 'bitfinex::DASH_BTC',
 'bitfinex::DASH_USD',
 'bitfinex::EDO_USD',
 'bitfinex::EOS_BTC',
 'bitfinex::EOS_EUR',
 'bitfinex::EOS_GBR',
 'bitfinex::EOS_JPY',


# Compare close prices / returns from Binance

## Load the data

The code below can be used to load all the existing data from two vendors CDD and CCXT. Current version is specified to Binance only, however, even for one exchange there's too many data to operate, that's why the output is the intersection of currency pairs between to universe, since one can compare only the intersection of currency pairs for two vendors.

In [9]:
# Load Binance-specific universe for CCXT.
ccxt_binance_universe = [
    element for element in ccxt_universe if element.startswith("binance")
]
# Load Binnance-specific universe for CDD.
cdd_binance_universe_initial = [
    element for element in universe_cdd if element.startswith("binance")
]
cdd_binance_universe = cdd_binance_universe_initial.copy()
# SCU_USDT has incorrect columns, so can not be downloaded.
# See CMTask244 - Cannot load CDD - binance - SCU/USDT from s3 for the reference.
cdd_binance_universe.remove("binance::SCU_USDT")
# The intersection of Binance currency pairs from two universes.
currency_pair_intersection_binance = set(ccxt_binance_universe).intersection(
    cdd_binance_universe_initial
)

In [10]:
root_dir = os.path.join(hs3.get_path(), "data")

In [11]:
cdd_data = []
cdd_loader = imcdalolo.CddLoader(root_dir=root_dir, aws_profile="am")

for full_symbol in currency_pair_intersection_binance:
    _, currency_pair = ivcdclcl.parse_full_symbol(full_symbol)
    cur_data = cdd_loader.read_data_from_filesystem(
        exchange_id="binance", currency_pair=currency_pair, data_type="ohlcv"
    )
    cdd_data.append(cur_data)
cdd_binance_df = pd.concat(cdd_data)

Reading CDD data for exchange id='binance', currencies='SOL_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/SOL_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='SOL_USDT'...
Index length increased by 270 = 129252 - 128982
Reading CDD data for exchange id='binance', currencies='EOS_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/EOS_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='EOS_USDT'...
Index length increased by 5289 = 888096 - 882807
Reading CDD data for exchange id='binance', currencies='ADA_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/ADA_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='ADA_USDT'...
Index length increased by 5288 = 854976 - 849688
Reading CDD data for exchange id='binance', currencies='BNB_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/BNB_USDT.csv.gz'..

In [12]:
display(cdd_binance_df.head(3))
display(cdd_binance_df.shape)

,open,high,low,close,volume,epoch,currency_pair,exchange_id
2021-06-06 06:57:00+00:00,39.654,39.709,39.592,39.649,3053.129,1.622963e+12,SOL/USDT,binance
2021-06-06 06:58:00+00:00,39.646,39.719,39.633,39.634,874.508,1.622963e+12,SOL/USDT,binance
2021-06-06 06:59:00+00:00,39.634,39.686,39.624,39.658,906.122,1.622963e+12,SOL/USDT,binance


(5599808, 8)

In [13]:
ccxt_client = imvcdclcl.CcxtCsvFileSystemClient(
    data_type="ohlcv", root_dir=root_dir, aws_profile="am"
)
multiple_symbols_client = ivcdclcl.MultipleSymbolsClient(
    class_=ccxt_client, mode="concat"
)
ccxt_binance_df = multiple_symbols_client.read_data(
    currency_pair_intersection_binance
)

Reading CCXT data for exchange id='binance', currencies='ADA_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/ADA_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='ADA_USDT'...
Index length increased by 4520 = 1622000 - 1617480
Reading CCXT data for exchange id='binance', currencies='BNB_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BNB_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='BNB_USDT'...
Index length increased by 4520 = 1622295 - 1617775
Reading CCXT data for exchange id='binance', currencies='BTC_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BTC_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='BTC_USDT'...
Index length increased by 4519 = 1619641 - 1615122
Reading CCXT data for exchange id='binance', currencies='EOS_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/EOS_USDT.csv.gz'...
Processing CCXT data for exchange id='binanc

In [14]:
ccxt_binance_df = ccxt_binance_df.sort_index()

In [15]:
display(ccxt_binance_df.head(3))
display(ccxt_binance_df.shape)

,full_symbol,open,high,low,close,volume,epoch,currency_pair,exchange_id
2018-08-17 00:00:00+00:00,binance::ADA_USDT,0.0946,0.0948,0.09442,0.09479,41334.200000,1.534464e+12,ADA_USDT,binance
2018-08-17 00:00:00+00:00,binance::BTC_USDT,6316.0000,6319.0400,6310.32000,6311.64000,9.967395,1.534464e+12,BTC_USDT,binance
2018-08-17 00:00:00+00:00,binance::BNB_USDT,9.7779,9.7791,9.75380,9.77780,520.660000,1.534464e+12,BNB_USDT,binance


(10084929, 9)

## Calculate returns and correlation

In [16]:
# CDD names cleaning.
cdd_binance_df["currency_pair"] = cdd_binance_df["currency_pair"].str.replace(
    "/", "_"
)

In [17]:
def resample_close_price(df: pd.DataFrame, resampling_freq: str) -> pd.Series:
    """
    Transform OHLCV data to the grouped series with resampled frequency and
    last close prices.

    :param df: OHLCV data
    :param resampling_freq: frequency from `pd.date_range()` to resample to
    :return: grouped and resampled close prices
    """
    # Reseting DateTime index, since pd.Grouper can't use index values.
    df = df.reset_index().rename(columns={"index": "stamp"})
    # Group by currency pairs and simultaneously resample to the desired frequency.
    resampler = df.groupby(
        ["currency_pair", pd.Grouper(key="stamp", freq=resampling_freq)]
    )
    # Take the last close value from each resampling period.
    close_series = resampler.close.last()
    return close_series

In [18]:
def calculate_correlations(
    ccxt_close_price: pd.Series, cdd_close_price: pd.Series, compute_returns: bool
) -> pd.DataFrame:
    """
    Take two series with close prices(i.e. CDD and CCXT data) and calculate the
    correlations for each specific currency pair.

    :param ccxt_series: grouped and resampled close prices for CCXT
    :param cdd_series: grouped and resampled close prices for CDD
    :param compute_returns: if True - compare returns, if False - compare close prices
    :return: grouped correlation matrix
    """
    if compute_returns:
        # Group by currency pairs in order to calculate the percentage returns.
        grouper_cdd = cdd_close_price.groupby("currency_pair")
        cdd_close_price = grouper_cdd.pct_change()
        grouper_ccxt = ccxt_close_price.groupby("currency_pair")
        ccxt_close_price = grouper_ccxt.pct_change()
    # Combine and calculate correlations.
    combined = pd.merge(
        cdd_close_price, ccxt_close_price, left_index=True, right_index=True
    )
    # Rename the columns.
    if compute_returns:
        combined.columns = ["ccxt_returns", "cdd_returns"]
    else:
        combined.columns = ["cdd_close", "ccxt_close"]
    # Group by again to calculte returns correlation for each currency pair.
    corr_matrix = combined.groupby(level=0).corr()
    return corr_matrix

In [19]:
# Corresponding resampled Series.
ccxt_binance_series_1d = resample_close_price(ccxt_binance_df, "1D")
cdd_binance_series_1d = resample_close_price(cdd_binance_df, "1D")

ccxt_binance_series_5min = resample_close_price(ccxt_binance_df, "5min")
cdd_binance_series_5min = resample_close_price(cdd_binance_df, "5min")

### 1-day returns

In [20]:
returns_corr_1day = calculate_correlations(
    ccxt_binance_series_1d, cdd_binance_series_1d, compute_returns=True
)
display(returns_corr_1day)

ccxt_returns  cdd_returns
currency_pair                                        
ADA_USDT      ccxt_returns      1.000000     0.997807
              cdd_returns       0.997807     1.000000
BNB_USDT      ccxt_returns      1.000000     0.998490
              cdd_returns       0.998490     1.000000
BTC_USDT      ccxt_returns      1.000000     0.997763
              cdd_returns       0.997763     1.000000
EOS_USDT      ccxt_returns      1.000000     0.998294
              cdd_returns       0.998294     1.000000
ETH_USDT      ccxt_returns      1.000000     0.995563
              cdd_returns       0.995563     1.000000
LINK_USDT     ccxt_returns      1.000000     0.997200
              cdd_returns       0.997200     1.000000
SOL_USDT      ccxt_returns      1.000000     0.996158
              cdd_returns       0.996158     1.000000

### 5-min returns

In [21]:
returns_corr_5min = calculate_correlations(
    ccxt_binance_series_5min, cdd_binance_series_5min, compute_returns=True
)
display(returns_corr_5min)

ccxt_returns  cdd_returns
currency_pair                                        
ADA_USDT      ccxt_returns      1.000000     0.986912
              cdd_returns       0.986912     1.000000
BNB_USDT      ccxt_returns      1.000000     0.987292
              cdd_returns       0.987292     1.000000
BTC_USDT      ccxt_returns      1.000000     0.990746
              cdd_returns       0.990746     1.000000
EOS_USDT      ccxt_returns      1.000000     0.993959
              cdd_returns       0.993959     1.000000
ETH_USDT      ccxt_returns      1.000000     0.974821
              cdd_returns       0.974821     1.000000
LINK_USDT     ccxt_returns      1.000000     0.983939
              cdd_returns       0.983939     1.000000
SOL_USDT      ccxt_returns      1.000000     0.999998
              cdd_returns       0.999998     1.000000

## Compare close prices

In [22]:
close_corr_1day = calculate_correlations(
    ccxt_binance_series_1d, cdd_binance_series_1d, compute_returns=False
)
display(close_corr_1day)

cdd_close  ccxt_close
currency_pair                                  
ADA_USDT      cdd_close    1.000000    0.999995
              ccxt_close   0.999995    1.000000
BNB_USDT      cdd_close    1.000000    0.999999
              ccxt_close   0.999999    1.000000
BTC_USDT      cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000
EOS_USDT      cdd_close    1.000000    0.999906
              ccxt_close   0.999906    1.000000
ETH_USDT      cdd_close    1.000000    0.999994
              ccxt_close   0.999994    1.000000
LINK_USDT     cdd_close    1.000000    0.999995
              ccxt_close   0.999995    1.000000
SOL_USDT      cdd_close    1.000000    0.999474
              ccxt_close   0.999474    1.000000

# Statistical properties of a full symbol in CDD

In [23]:
# Clearing CDD currency pairs that are incorrect.

# Binance
universe_cdd.remove("binance::SCU_USDT")

# Bitfinex
universe_cdd.remove("bitfinex::BTC_GBR")  # doesn't exist.
universe_cdd.remove("bitfinex::DASH_BTC")  # NaT in stamps.
universe_cdd.remove("bitfinex::DASH_USD")  # NaT in stamps.
universe_cdd.remove("bitfinex::EOS_GBR")  # doesn't exist.
universe_cdd.remove("bitfinex::ETH_GBR")  # doesn't exist.
universe_cdd.remove("bitfinex::NEO_GBR")  # doesn't exist.
universe_cdd.remove("bitfinex::QTUM_USD")  # NaT in stamps.
universe_cdd.remove("bitfinex::TRX_GBR")  # doesn't exist.
universe_cdd.remove("bitfinex::XLM_GBR")  # doesn't exist.

# ftx has some critical mistakes in the downloading process, so can not continue analysis with them.
# see CMTask801 - Downloading issues of FTX exchange from CDD universe for further reference.
cdd_ftx_universe = [
    element for element in universe_cdd if element.startswith("ftx")
]
for elem in cdd_ftx_universe:
    universe_cdd.remove(elem)
    
# kucoin exchange: the timestamps are obviously wrong and with too short time period.
# see CMTask253 - Fix timestamp for CDD - kucoin for reference.
cdd_kucoin_universe = [
    element for element in universe_cdd if element.startswith("kucoin")
]
for elem in cdd_kucoin_universe:
    universe_cdd.remove(elem)

In [24]:
# TODO(Max): consider using compute_start_end_table function
def calculate_statistics_for_stamps(coin_list: List[str], vendor: str) -> pd.DataFrame:
    """
    Load the OHLCV data for each currency pair in CDD or CCXT universe and
    compute the corresponding descriptive statistics.
    Stats include:
        - index - currency pair
        - exchange_id - exchange_id
        - data_points_counts - number of timestamps for each coin
        - NaNs_in_Close - number of NaNs in "close"
        - step_in_stamp - value counts of steps between timestamps
        - start_date
        - end_date
    
    :param coin_list: list of all full symbols in CDD or CCXT universe
    :param vendor: CCXT or CDD 
    :return: descriptive statistics for each full symbol
    """
    # Load data for each full symbol.
    result = []
    cdd_loader = imcdalolo.CddLoader(root_dir=root_dir, aws_profile="am")
    for full_symbol in coin_list:
        if vendor == "cdd":
            exchange_id, currency_pair = ivcdclcl.parse_full_symbol(full_symbol)
            coin = cdd_loader.read_data_from_filesystem(
                exchange_id=exchange_id,
                currency_pair=currency_pair,
                data_type="ohlcv",
            )
        else:
            ccxt_client = imvcdclcl.CcxtCsvFileSystemClient(
                data_type="ohlcv", root_dir=root_dir, aws_profile="am"
            )
            multiple_symbols_client = ivcdclcl.MultipleSymbolsClient(
                class_=ccxt_client, mode="concat"
            )
            coin = multiple_symbols_client.read_data([full_symbol])
            exchange_id, currency_pair = ivcdclcl.parse_full_symbol(full_symbol)
            coin = coin.sort_index()
        # Reseting DateTime index, so it can be further used in the calculations.
        coin.reset_index(inplace=True)
        coin = coin.rename(columns={"index": "stamp"})
        # The value of the step between two data points.
        stamp_steps = pd.Series(coin["stamp"].diff().value_counts().index)
        # Start-end date.
        max_date = pd.Series(
            coin["stamp"].describe(datetime_is_numeric=True).loc["max"]
        )
        min_date = pd.Series(
            coin["stamp"].describe(datetime_is_numeric=True).loc["min"]
        )
        # Number of timestamps for each coin.
        data_points = pd.Series(
            coin["stamp"].describe(datetime_is_numeric=True).loc["count"]
        )
        # Attach calculations to the DataFrame.
        stamp_stats = pd.DataFrame()
        stamp_stats["exchange_id"] = [exchange_id]
        stamp_stats["data_points_counts"] = data_points
        stamp_stats["NaNs_in_Close"] = len(coin[coin["close"].isna()])
        stamp_stats["step_in_stamp"] = stamp_steps
        stamp_stats["start_date"] = min_date
        stamp_stats["end_date"] = max_date
        stamp_stats.index = [currency_pair]
        result.append(stamp_stats)
    result = pd.concat(result)
    if vendor == "cdd":
        result = result.add_suffix("_cdd")
    else:
        result = result.add_suffix("_ccxt")
    return result

## Comparison of intersection of full symbols between CCXT and CDD

In [25]:
# Full symbols that are included in CDD but not in CCXT (cleaned from unavailable full symbols).
cdd_and_ccxt_cleaned = set(ccxt_universe).intersection(universe_cdd)
len(cdd_and_ccxt_cleaned)

7

In [26]:
# Load the intersection of full symbols for CDD and CCXT.
stats_for_stamps_ccxt = calculate_statistics_for_stamps(
    cdd_and_ccxt_cleaned, vendor="ccxt"
)
stats_for_stamps_cdd_union = calculate_statistics_for_stamps(
    cdd_and_ccxt_cleaned, vendor="cdd"
)

Reading CCXT data for exchange id='binance', currencies='SOL_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/SOL_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='SOL_USDT'...
Index length increased by 1224 = 576140 - 574916
Reading CCXT data for exchange id='binance', currencies='EOS_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/EOS_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='EOS_USDT'...
Index length increased by 4520 = 1622444 - 1617924
Reading CCXT data for exchange id='binance', currencies='ADA_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/ADA_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='ADA_USDT'...
Index length increased by 4520 = 1622000 - 1617480
Reading CCXT data for exchange id='binance', currencies='BNB_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BNB_USDT.csv.gz'...
Processing CCXT data for exchange id='binance'

In [27]:
union_cdd_ccxt_stats = pd.concat(
    [stats_for_stamps_cdd_union, stats_for_stamps_ccxt], axis=1
)
union_cdd_ccxt_stats.sort_values("exchange_id_cdd")

,exchange_id_cdd,data_points_counts_cdd,NaNs_in_Close_cdd,step_in_stamp_cdd,start_date_cdd,end_date_cdd,exchange_id_ccxt,data_points_counts_ccxt,NaNs_in_Close_ccxt,step_in_stamp_ccxt,start_date_ccxt,end_date_ccxt
SOL_USDT,binance,129252,270,0 days 00:01:00,2021-06-06 06:57:00+00:00,2021-09-04 01:08:00+00:00,binance,576140,1224,0 days 00:01:00,2020-08-11 06:00:00+00:00,2021-09-15 08:19:00+00:00
EOS_USDT,binance,888096,5289,0 days 00:01:00,2020-01-08 08:00:00+00:00,2021-09-16 01:35:00+00:00,binance,1622444,4520,0 days 00:01:00,2018-08-17 00:00:00+00:00,2021-09-16 16:43:00+00:00
ADA_USDT,binance,854976,5288,0 days 00:01:00,2020-01-31 08:01:00+00:00,2021-09-16 01:36:00+00:00,binance,1622000,4520,0 days 00:01:00,2018-08-17 00:00:00+00:00,2021-09-16 09:19:00+00:00
BNB_USDT,binance,840575,5289,0 days 00:01:00,2020-02-10 08:01:00+00:00,2021-09-16 01:35:00+00:00,binance,1622295,4520,0 days 00:01:00,2018-08-17 00:00:00+00:00,2021-09-16 14:14:00+00:00
BTC_USDT,binance,1063179,5289,0 days 00:01:00,2019-09-08 17:57:00+00:00,2021-09-16 01:35:00+00:00,binance,1619641,4519,0 days 00:01:00,2018-08-17 00:00:00+00:00,2021-09-14 18:00:00+00:00
LINK_USDT,binance,875139,5289,0 days 00:01:00,2020-01-17 08:00:00+00:00,2021-09-16 01:38:00+00:00,binance,1402520,3890,0 days 00:01:00,2019-01-16 10:00:00+00:00,2021-09-16 09:19:00+00:00
ETH_USDT,binance,948591,5288,0 days 00:01:00,2019-11-27 07:45:00+00:00,2021-09-16 01:35:00+00:00,binance,1619889,4520,0 days 00:01:00,2018-08-17 00:00:00+00:00,2021-09-14 22:08:00+00:00


## Comparison of full symbols that are included in CDD but not available in CCXT

In [28]:
# Set of full symbols that are included in CDD but not available in CCXT (cleaned from unavailable full symbols).
cdd_and_not_ccxt_cleaned = set(universe_cdd).difference(ccxt_universe)
len(cdd_and_not_ccxt_cleaned)

70

In [29]:
# Clean the set from non-USDT pairs. 
cdd_and_not_ccxt_cleaned_usdt = [
    element for element in cdd_and_not_ccxt_cleaned if element.endswith("USDT")
]
len(cdd_and_not_ccxt_cleaned_usdt)

29

In [30]:
stats_for_stamps_cdd = calculate_statistics_for_stamps(
    cdd_and_not_ccxt_cleaned_usdt, vendor="cdd"
)

Reading CDD data for exchange id='binance', currencies='VET_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/VET_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='VET_USDT'...
Index length increased by 270 = 129252 - 128982
Reading CDD data for exchange id='binance', currencies='LTC_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/LTC_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='LTC_USDT'...
Index length increased by 5288 = 886647 - 881359
Reading CDD data for exchange id='binance', currencies='AAVE_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/AAVE_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='AAVE_USDT'...
Index length increased by 270 = 129252 - 128982
Reading CDD data for exchange id='binance', currencies='ICP_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/ICP_USDT.csv.gz'

Currently there are the following descriptive statistics:

- index - currency pair
- exchange_id - exchange_id
- data_points_counts - number of timestamps for each coin
- NaNs_in_Close - number of NaNs in "close"
- step_in_stamp - value counts of steps between timestamps
- start_date
- end_date

What else can be added here?

In [31]:
stats_for_stamps_cdd.sort_values("exchange_id_cdd")

,exchange_id_cdd,data_points_counts_cdd,NaNs_in_Close_cdd,step_in_stamp_cdd,start_date_cdd,end_date_cdd
VET_USDT,binance,129252,270,0 days 00:01:00,2021-06-06 06:57:00+00:00,2021-09-04 01:08:00+00:00
ICX_USDT,binance,113356,270,0 days 00:01:00,2021-06-06 07:24:00+00:00,2021-08-24 00:39:00+00:00
TRX_USDT,binance,878012,5289,0 days 00:01:00,2020-01-15 08:05:00+00:00,2021-09-16 01:36:00+00:00
NEO_USDT,binance,830452,5289,0 days 00:01:00,2020-02-17 08:45:00+00:00,2021-09-16 01:36:00+00:00
BTT_USDT,binance,531656,5288,0 days 00:01:00,2020-09-11 20:41:00+00:00,2021-09-16 01:36:00+00:00
ZEC_USDT,binance,847776,5289,0 days 00:01:00,2020-02-05 08:01:00+00:00,2021-09-16 01:36:00+00:00
DAI_USDT,binance,1000,0,0 days 00:01:00,2020-08-11 11:20:00+00:00,2020-08-12 03:59:00+00:00
DASH_USDT,binance,849211,5289,0 days 00:01:00,2020-02-04 08:06:00+00:00,2021-09-16 01:36:00+00:00
USDC_USDT,binance,531656,5288,0 days 00:01:00,2020-09-11 20:41:00+00:00,2021-09-16 01:36:00+00:00
PAX_USDT,binance,517399,5288,0 days 00:01:00,2020-09-11 20:41:00+00:00,2021-09-06 03:59:00+00:00
